In [1]:
from Util.Counting_and_Detection import findRoot
import cv2
import torch
from Model.pose_higher_hrnet import PoseHigherResolutionNet as get_model_HR
from Detection_Torch.input_param import cfg
from Detection_Torch.input_param import args as args
from glob import glob
import os
import json
import numpy as np
from sklearn.neighbors import KDTree
from Util.Counting_and_Detection import findRoot
from Util.Matching import Find_Correspondence_Using_Hist, Find_Correspondence_Using_Dist
from Util.Counting_and_Detection import predict_strand_num, computeSearchDist


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
No dq3d python package, filterreg deformation model not available.
F:\cuda10\bin;C:\Users\User\.conda\envs\hair_proj;C:\Users\User\.conda\envs\hair_proj\Library\mingw-w64\bin;C:\Users\User\.conda\envs\hair_proj\Library\usr\bin;C:\Users\User\.conda\envs\hair_proj\Library\bin;C:\Users\User\.conda\envs\hair_proj\Scripts;C:\Users\User\.conda\envs\hair_proj\bin;C:\ProgramData\Anaconda3\condabin;C:\ProgramData\Anaconda3;C:\ProgramData\Anaconda3\Library\mingw-w64\bin;C:\ProgramData\Anaconda3\Library\usr\bin;C:\ProgramData\Anaconda3\Library\bin;C:\ProgramData\Anaconda3\Scripts;F:\cuda10\bin;F:\cuda10\libnvvp;F:\cuda\bin;F:\cuda\libnvvp;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0;C:\Program Files (x86)\NVIDIA Corporati

In [2]:
rootDetectionModel = get_model_HR(cfg).cuda()
weights_dir = args.weights_dir_HRNet
rootDetectionModel.load_state_dict(torch.load(weights_dir), strict=False) 

<All keys matched successfully>

In [3]:
def nms(cur_img_dot_list):
    shape=[480,640]
    cur_target_pt = np.array(cur_img_dot_list,dtype="float32")
    sizeY, sizeX = shape
    _, cur_target_pt, _, _, _ = computeSearchDist(cur_target_pt, sizeY, sizeX)
    cur_target_pt = cur_target_pt[:, :2]
    cur_img_dot_list = cur_target_pt.astype(int).tolist()
    return cur_img_dot_list

In [4]:
use_cuda = True
INIT_FRAME =0
STITCHING_TERM = 1
# For deep learning
IMAGE_SCALE_RATIO = 1.0
MAX_ITER_NUM = 100 # 30 # 
INIT_DOT_INDEX = None 
outlier_ratio = 0.35 

CHECK_ALL_RANGE = True # False # 

HIST_ANGLE_NUM = 18
HIST_RATIO_NUM = 20
RANDOM_INIT_DOT_SELECTION = False
INIT_DOT_ITERATION_NUM = 10
K_NUM_FOR_CORRESPONDENCE = 4 # 

USING_DISTANCE_FILTER = False # True

stitching_term = STITCHING_TERM
PRINT_INFO = False
PRINT_TIME_INFO = False
DO_DRAWING = False
DO_MATCHING = True
DO_STITCHING = True
DO_COUNT = True
BINARY_SCORE = True
INIT_CANDIDATE_SORT_ORDER = 'min'
DISTANCE_FILTER = USING_DISTANCE_FILTER

In [5]:
IMG_FOLDER = "data"
IMG_FORMAT = ".jpg"
AUGMENT_DATA = True

HEIGHT=480
WIDTH=640

In [6]:
dir_list = glob(os.path.join(IMG_FOLDER,"*"))

In [7]:
def stiching(prev_img_dot_list, cur_img_dot_list, cur_img_path):
    #prev -> not 움직임
    #cur -> 움직임
    next_img = cv2.imread(cur_img_path)
    prev_img = cv2.imread(cur_img_path)
    target_pt, source_pt, _, transformed_target_pt, tf_param = Find_Correspondence_Using_Hist(next_img, prev_img, 
                                    [cur_img_dot_list, prev_img_dot_list], 
                                    BINARY_SCORE = BINARY_SCORE,
                                    HIST_ANGLE_NUM = HIST_ANGLE_NUM, HIST_RATIO_NUM = HIST_RATIO_NUM,
                                    MAX_ITER_NUM = MAX_ITER_NUM, CHECK_ALL_RANGE = CHECK_ALL_RANGE,
                                    outlier_ratio = outlier_ratio, 
                                    INIT_DOT_ITERATION_NUM = INIT_DOT_ITERATION_NUM, 
                                    K_NUM_FOR_CORRESPONDENCE = K_NUM_FOR_CORRESPONDENCE,
                                    RANDOM_INIT_DOT_SELECTION = RANDOM_INIT_DOT_SELECTION, 
                                    INIT_CANDIDATE_SORT_ORDER = INIT_CANDIDATE_SORT_ORDER,
                                    INIT_DOT_INDEX = INIT_DOT_INDEX,
                                    DISTANCE_FILTER = DISTANCE_FILTER,
                                    use_cuda=use_cuda)       
    if transformed_target_pt is None:
        target_pt, source_pt, _, transformed_target_pt, tf_param = Find_Correspondence_Using_Dist(next_img, prev_img, 
                                    [cur_img_dot_list, prev_img_dot_list], 
                                    BINARY_SCORE = BINARY_SCORE,
                                    HIST_ANGLE_NUM = HIST_ANGLE_NUM, HIST_RATIO_NUM = HIST_RATIO_NUM,
                                    MAX_ITER_NUM = MAX_ITER_NUM, CHECK_ALL_RANGE = CHECK_ALL_RANGE,
                                        outlier_ratio = outlier_ratio, 
                                    INIT_DOT_ITERATION_NUM = INIT_DOT_ITERATION_NUM, 
                                    K_NUM_FOR_CORRESPONDENCE = K_NUM_FOR_CORRESPONDENCE,
                                    RANDOM_INIT_DOT_SELECTION = RANDOM_INIT_DOT_SELECTION, 
                                    INIT_CANDIDATE_SORT_ORDER = INIT_CANDIDATE_SORT_ORDER,
                                    INIT_DOT_INDEX = INIT_DOT_INDEX,
                                    DISTANCE_FILTER = DISTANCE_FILTER,
                                    use_cuda=use_cuda)
    return transformed_target_pt

In [8]:
def augment(index, img_path_list):
    dot_json_path = os.path.join(series_path, "json_to_txt", os.path.basename(img_path_list[index]).replace(IMG_FORMAT,".json"))

    dot_path = os.path.join(series_path, "json_to_txt", "dot", os.path.basename(img_path_list[index]).replace(IMG_FORMAT,".txt"))
    cls_path = os.path.join(series_path, "json_to_txt", "cls", os.path.basename(img_path_list[index]).replace(IMG_FORMAT,".txt"))
    cur_dot_list = nms(findRoot(cv2.imread(img_path_list[index]), rootDetectionModel, ADDITIONAL_IMG = False))

    augment_labels_arr = []
    augment_points_arr = []
    augment_points_set = set()
    def prev_augment():
        pre_json_path = img_path_list[index-1].replace(IMG_FORMAT, ".json")
        with open(pre_json_path) as f:
            json_dict = json.load(f)    

        labels_arr = []
        point_arr = []
        for item in json_dict["shapes"]:
            label = item["label"]
            labels_arr.append(label)
            point_arr.append(item["points"][0])
        transformed_cur_pt = stiching(point_arr, cur_dot_list, img_path_list[index])
        tree = KDTree(point_arr, leaf_size=2)

        if transformed_cur_pt is not None:
            for i in range(len(cur_dot_list)):
                if i in augment_points_set:
                    continue
                query_result =  tree.query_radius([transformed_cur_pt[i]],r=10)[0]
                if(len(query_result)==0): continue

                augment_points_set.add(i)
                nearest_point_idx = query_result[0]
                augment_labels_arr.append(labels_arr[nearest_point_idx])
                augment_points_arr.append(cur_dot_list[i])

    def next_augment():
        next_json_path = img_path_list[index+1].replace(IMG_FORMAT, ".json")
        with open(next_json_path) as f:
            json_dict = json.load(f)    

        labels_arr = []
        point_arr = []
        for item in json_dict["shapes"]:
            label = item["label"]
            labels_arr.append(label)
            point_arr.append(item["points"][0])
        transformed_cur_pt = stiching(point_arr, cur_dot_list, img_path_list[index])
        tree = KDTree(point_arr, leaf_size=2)
        
        if transformed_cur_pt is not None:
            for i in range(len(cur_dot_list)):
                if i in augment_points_set:
                    continue
                augment_points_set.add(i)
                query_result =  tree.query_radius([transformed_cur_pt[i]],r=10)[0]
                if(len(query_result)==0): continue

                augment_points_set.add(i)
                nearest_point_idx = query_result[0]
                augment_labels_arr.append(labels_arr[nearest_point_idx])
                augment_points_arr.append(cur_dot_list[i])
            
    prev_augment()
    next_augment()
    with open(dot_path, "w") as txt_file:
        for line in augment_points_arr:
            line = str(int(line[0]))+" "+str(int(line[1]))
            txt_file.write(line + "\n")

    with open(cls_path, "w") as txt_file:
        for line in augment_labels_arr:
            txt_file.write(line + "\n")

    shape_arr = []
    for i in range(len(augment_points_arr)):
        shape_dict = {
            "label": str(augment_labels_arr[i]),
            "points": [[int(augment_points_arr[i][0]),int(augment_points_arr[i][1])]],
            "group_id": None,
            "shape_type": "point",
            "flags": {}
        }
        shape_arr.append(shape_dict)

    result_dict = {
        "version": "4.5.9",
        "flags": {},
        "shapes": shape_arr,
        "imagePath": os.path.basename(img_path),
        "imageData": None,
        "imageHeight": HEIGHT,
        "imageWidth": WIDTH
    }
    with open(dot_json_path, 'w') as f:
        json.dump(result_dict, f, indent=4)

In [13]:
for series_path in dir_list:
    img_path_list = glob(os.path.join(series_path,"*"+IMG_FORMAT))
    img_path_list = sorted(img_path_list)
    series_id = os.path.basename(series_path)
    if("." in series_path):
        continue
    if os.path.exists(os.path.join(os.getcwd(), IMG_FOLDER, series_id, "json_to_txt")):
        print(series_path, "result dir exists")
        continue
    else:
        os.mkdir(os.path.join(os.getcwd(), IMG_FOLDER, series_id, "json_to_txt"))
        os.mkdir(os.path.join(os.getcwd(), IMG_FOLDER, series_id, "json_to_txt", "dot"))
        os.mkdir(os.path.join(os.getcwd(), IMG_FOLDER, series_id, "json_to_txt", "cls"))
    for i, img_path in enumerate(img_path_list):
        json_path = img_path.replace(IMG_FORMAT, ".json")
        
        dot_path = os.path.join(series_path, "json_to_txt", "dot", os.path.basename(img_path).replace(IMG_FORMAT,".txt"))
        cls_path = os.path.join(series_path, "json_to_txt", "cls", os.path.basename(img_path).replace(IMG_FORMAT,".txt"))
        
        if(os.path.exists(json_path)):
            with open(json_path) as f:
                json_dict = json.load(f)
        else:
            augment(i, img_path_list)
            continue
        labels_arr = []
        point_arr = []
        for item in json_dict["shapes"]:
            label = item["label"]
            point = str(int(item["points"][0][0]))+" "+str(int(item["points"][0][1]))
            labels_arr.append(label)
            point_arr.append(point)

        with open(dot_path, "w") as txt_file:
            for line in point_arr:
                txt_file.write(line + "\n")

        with open(cls_path, "w") as txt_file:
            for line in labels_arr:
                txt_file.write(line + "\n")